<a href="https://colab.research.google.com/github/keepRunning1204/data-science/blob/main/Preprocessing_Stroke_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing
import plotly.express as px #visualization
from plotly.subplots import make_subplots #visualization
from matplotlib import pyplot as plt #visualization
import seaborn as sns #visualization
from sklearn import preprocessing #label encoding
from imblearn.over_sampling import SMOTE #oversample data
import plotly.graph_objects as go #visualization
from sklearn.model_selection import train_test_split #split data

In [3]:
data=pd.read_csv('/content/drive/MyDrive/healthcare-dataset-stroke-data.csv')
data.head

<bound method NDFrame.head of          id  gender   age  hypertension  heart_disease ever_married  \
0      9046    Male  67.0             0              1          Yes   
1     51676  Female  61.0             0              0          Yes   
2     31112    Male  80.0             0              1          Yes   
3     60182  Female  49.0             0              0          Yes   
4      1665  Female  79.0             1              0          Yes   
...     ...     ...   ...           ...            ...          ...   
5105  18234  Female  80.0             1              0          Yes   
5106  44873  Female  81.0             0              0          Yes   
5107  19723  Female  35.0             0              0          Yes   
5108  37544    Male  51.0             0              0          Yes   
5109  44679  Female  44.0             0              0          Yes   

          work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0           Private          Urban   

In [4]:
data.shape

(5110, 12)

In [5]:
data.info

<bound method DataFrame.info of          id  gender   age  hypertension  heart_disease ever_married  \
0      9046    Male  67.0             0              1          Yes   
1     51676  Female  61.0             0              0          Yes   
2     31112    Male  80.0             0              1          Yes   
3     60182  Female  49.0             0              0          Yes   
4      1665  Female  79.0             1              0          Yes   
...     ...     ...   ...           ...            ...          ...   
5105  18234  Female  80.0             1              0          Yes   
5106  44873  Female  81.0             0              0          Yes   
5107  19723  Female  35.0             0              0          Yes   
5108  37544    Male  51.0             0              0          Yes   
5109  44679  Female  44.0             0              0          Yes   

          work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0           Private          Urban 

In [6]:
#statistical description of data numeric columns
data.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [7]:
#drop Other value
data.drop(data[data.gender == 'Other'].index, inplace=True)
#drop id column
data.drop('id', axis=1, inplace=True)
#columns to convert to category
cols = ['gender', 'ever_married', 'work_type','Residence_type','smoking_status']
#convert columns to category
data[cols] = data[cols].astype('category')
#convert age column to int
data['age'] = data['age'].astype('int')

#check columns types
print(data.dtypes)

gender               category
age                     int64
hypertension            int64
heart_disease           int64
ever_married         category
work_type            category
Residence_type       category
avg_glucose_level     float64
bmi                   float64
smoking_status       category
stroke                  int64
dtype: object


In [8]:
data.isnull().sum().sort_values(ascending=False)

bmi                  201
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
smoking_status         0
stroke                 0
dtype: int64

In [9]:
#creating a copy of dataset for the visualization
eda= data.copy()
#change values 1, 0 to have the issue or not
eda['hypertension'] = np.where(eda['hypertension'] == 1, 'Hypertension', 'No hypertension')
eda['heart_disease'] = np.where(eda['heart_disease'] == 1, 'Heart disease', 'No heart disease')
eda['stroke'] = np.where(eda['stroke'] == 1, 'Stroke', 'No stroke')
eda['ever_married'] = np.where(eda['ever_married'] == 'Yes', 'Ever married', 'Never')

eda.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67,No hypertension,Heart disease,Ever married,Private,Urban,228.69,36.6,formerly smoked,Stroke
1,Female,61,No hypertension,No heart disease,Ever married,Self-employed,Rural,202.21,NaN,never smoked,Stroke
2,Male,80,No hypertension,Heart disease,Ever married,Private,Rural,105.92,32.5,never smoked,Stroke
3,Female,49,No hypertension,No heart disease,Ever married,Private,Urban,171.23,34.4,smokes,Stroke
4,Female,79,Hypertension,No heart disease,Ever married,Self-employed,Rural,174.12,24.0,never smoked,Stroke


In [10]:
#make subplots to create 3 visualizations in the same space
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Age", "Avg glucose level", "BMI"))

#first plot config
fig.add_trace(go.Histogram(x=eda.age[eda['stroke'] == 'Stroke'],
                           name='Stroke',marker=dict(color='#d5a036', line=dict(width=1.5, color='#000000')),
                           xbins=dict(size=10 )),row=1, col=1)
#first plot config
fig.add_trace(go.Histogram(x=eda.age[eda['stroke'] == 'No stroke'],
                           name='Normal',marker=dict(color='#1984c5', line=dict(width=1.5, color='#000000')),
                           xbins=dict(size=10 )),row=1, col=1)
#second plot config
fig.add_trace(go.Histogram(x=eda.avg_glucose_level[eda['stroke'] == 'Stroke'],
                           marker=dict(color='#d5a036', line=dict(width=1.5, color='#000000')),
                           showlegend=False,xbins=dict(size=25)),row=1, col=2)
#second plot config
fig.add_trace(go.Histogram(x=eda.avg_glucose_level[eda['stroke'] == 'No stroke'],
                           marker=dict(color='#1984c5', line=dict(width=1.5, color='#000000')),
                           showlegend=False,xbins=dict(size=25 )),row=1, col=2)
#third plot config
fig.add_trace(go.Histogram(x=eda.bmi[eda['stroke'] == 'Stroke'],
                           marker=dict(color='#d5a036', line=dict(width=1.5, color='#000000')),
                           showlegend=False,xbins=dict(size=5 )),row=1, col=3)
#third plot config
fig.add_trace(go.Histogram(x=eda.bmi[eda['stroke'] == 'No stroke'],
                           marker=dict(color='#1984c5', line=dict(width=1.5, color='#000000')),
                           showlegend=False,xbins=dict(size=5 )),row=1, col=3)
#Tittle config
fig.update_layout(title_x=0.5,title_text="Stroke distribution by:", height=400,barmode='stack',font_family='Verdana',
                  font=dict(family="Verdana,Verdana",size=12),paper_bgcolor='#edeae7',plot_bgcolor='#edeae7')
#config opacity
fig.update_traces(opacity=0.80)

fig.show()

In [11]:
#create variable with median age, median glucose and median bmi
median_age=eda.groupby('stroke')['age'].median().round(0).astype(int)
median_glucose=eda.groupby('stroke')['avg_glucose_level'].median().round(0).astype(int)
median_bmi=eda.groupby('stroke')['bmi'].median().round(0).astype(int)
#reset index and sort values
median_age=median_age.reset_index().sort_values('age', ascending=False)
median_glucose=median_glucose.reset_index().sort_values('avg_glucose_level', ascending=False)
median_bmi=median_bmi.reset_index().sort_values('bmi', ascending=False)
#change values to yes and no to stroke and normal
x_value = median_age.stroke

#make subplots to create 3 visualizations in the same space
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Age", "Avg glucose level", "BMI"))

#create bar plot

#first plot config
fig.add_trace(go.Bar(x=x_value,y=median_age.age, text=median_age.age,textposition='outside',showlegend=False,
                     marker=dict(color=['#d5a036', '#1984c5'],line=dict(width=1.5, color='#000000'), opacity=0.9)),row=1, col=1)

#second plot config
fig.add_trace(go.Bar(x=x_value,y=median_glucose.avg_glucose_level, text=median_glucose.avg_glucose_level,
                     showlegend=False, textposition='outside',
                     marker=dict(color=['#d5a036', '#1984c5'],line=dict(width=1.5, color='#000000'), opacity=0.9)),row=1, col=2)

#third plot config
fig.add_trace(go.Bar(x=x_value,y=median_bmi.bmi, text=median_bmi.bmi,textposition='outside',showlegend=False,
                     marker=dict(color=['#d5a036', '#1984c5'],line=dict(width=1.5, color='#000000'), opacity=0.9)),row=1, col=3)

#bar plot config
fig.update_layout(title_x=0.5,title_text="Median Values by:", height=450,font_family='Verdana',
                  font=dict(family="Verdana,Verdana",size=13),paper_bgcolor='#edeae7',plot_bgcolor='#edeae7')
#bar plot config
fig.update_traces(textfont_size=14, textangle=0, textposition="outside", cliponaxis=False,
                  marker_line_width=1.5,marker_line_color="black")

#config space subtitles
for i in range(0,3):
    fig.layout.annotations[i].update(y=1.05)

#bar plot config
fig.update_xaxes(showline=True, linewidth=1, linecolor='black')

fig.show()

In [12]:
#crate dataframe for variables with 2 values and 4 values
visu1 = pd.DataFrame()
visu4 = pd.DataFrame()
#variables with 0 for the iteration
num = 0
num1 = 0
#variables with the name of columns with 2 values and 4
variables2 = ['gender','hypertension','heart_disease','ever_married','Residence_type']
variables4 = ['work_type','smoking_status']
#for loop to get the percentage of columns
for i in variables2:
    num += 1
    temp = eda.groupby(i)['stroke'].value_counts(normalize=True)
    temp = temp.mul(100).rename('percentage' + str(num)).reset_index()
    visu1 = pd.concat([visu1, temp], axis=1)


for i in variables4:
    num1 += 1
    temp = eda.groupby(i)['stroke'].value_counts(normalize=True)
    temp = temp.mul(100).rename('percentage' + str(num1)).reset_index()
    visu4 = pd.concat([visu4, temp], axis=1)



#drop diplicated values
visu1 = visu1.loc[:, ~visu1.columns.duplicated()]
#round values with 2 decimals
visu1 = visu1.round(decimals = 2)

#drop diplicated values
visu4 = visu4.loc[:, ~visu4.columns.duplicated()]
#round values with 2 decimals
visu4 = visu4.round(decimals = 2)

#colors for the visualizations
color_rgb = ['rgb(25, 132, 197)','rgb(213, 160, 54)']
#variables with 0 for the iteration
num = 0
num1 = 0
#loop for create visualizations
for i in variables2:
    num +=1
    fig = px.histogram(visu1, x=i, y='percentage' + str(num),
                       color='stroke', barmode='group',
                       text_auto=True,
                       height=450, width = 600,color_discrete_sequence= color_rgb,opacity=0.8)
    #config plot
    fig.update_layout(title_x=0.5,title_text=(f"Stroke by {i}"), height=450,font_family='Verdana',
                      font=dict(family="Verdana,Verdana",size=13),yaxis_title=None, xaxis_title=None,
                      paper_bgcolor='#edeae7',plot_bgcolor='#edeae7')

    #config plot
    fig.update_traces(textfont_size=14, textangle=0, textposition="outside", cliponaxis=False,
                      marker_line_width=1,marker_line_color="black")
    fig.update_yaxes(title="", ticksuffix='%')
    fig.show()

#loop for create visualizations
for i in variables4:
    num1 +=1
    fig = px.histogram(visu4, x=i, y='percentage' + str(num1),
                       color='stroke', barmode='group',
                       text_auto=True,
                       height=450, width = 600,color_discrete_sequence= color_rgb,opacity=0.8)
    #config plot
    fig.update_layout(title_x=0.5,title_text=(f"Stroke by {i}"), height=450,font_family='Verdana',
                      font=dict(family="Verdana,Verdana",size=13),yaxis_title=None,
                      xaxis_title=None,paper_bgcolor='#edeae7',plot_bgcolor='#edeae7')

    #config plot
    fig.update_traces(textfont_size=14, textangle=0, textposition="outside", cliponaxis=False,
                      marker_line_width=1,marker_line_color="black")
    fig.update_yaxes(title="", ticksuffix='%')
    fig.show()

In [13]:
#均值填充 bmi
mean_bmi = data['bmi'].mean()
data['bmi'].fillna(mean_bmi, inplace=True)

In [14]:
data.isnull().sum().sort_values(ascending=False)

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [15]:
#Categorical Encoding is a process where we transform categorical data into numerical data.
#convert in binary columns with 2 results
columns_obj = ["gender", "ever_married" ,"Residence_type"]
encoding = preprocessing.LabelEncoder()
for col in columns_obj:
    data[col]=  encoding.fit_transform(data[col])
#convert in 0 and 1 the rest of columns
data = pd.get_dummies(data)
data.head()

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1,67,0,1,1,1,228.69,36.60000,1,0,0,1,0,0,0,1,0,0
1,0,61,0,0,1,0,202.21,28.89456,1,0,0,0,1,0,0,0,1,0
2,1,80,0,1,1,0,105.92,32.50000,1,0,0,1,0,0,0,0,1,0
3,0,49,0,0,1,1,171.23,34.40000,1,0,0,1,0,0,0,0,0,1
4,0,79,1,0,1,0,174.12,24.00000,1,0,0,0,1,0,0,0,1,0


In [16]:
data.corr()

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
gender,1.000000,-0.027855,0.021223,0.085685,-0.030171,-0.006105,0.054722,-0.025605,0.009081,-0.016598,0.011486,-0.032502,-0.026045,0.089226,0.059557,0.043121,-0.098952,0.011408
age,-0.027855,1.000000,0.276273,0.263649,0.679141,0.014019,0.238212,0.326189,0.245109,0.130036,-0.078560,0.117149,0.327861,-0.634997,-0.378774,0.237354,0.119479,0.073221
hypertension,0.021223,0.276273,1.000000,0.108292,0.164187,-0.007980,0.174540,0.160151,0.127891,0.017661,-0.021612,-0.005358,0.115417,-0.129535,-0.141551,0.059022,0.065018,0.031213
heart_disease,0.085685,0.263649,0.108292,1.000000,0.114601,0.003045,0.161907,0.038865,0.134905,0.001312,-0.015715,0.000068,0.086742,-0.091654,-0.066765,0.066938,-0.021893,0.044030
ever_married,-0.030171,0.679141,0.164187,0.114601,1.000000,0.005988,0.155329,0.335564,0.108299,0.133964,-0.090873,0.154104,0.192637,-0.544658,-0.332335,0.171960,0.102663,0.105132
Residence_type,-0.006105,0.014019,-0.007980,0.003045,0.005988,1.000000,-0.004783,-0.000288,0.015415,0.013056,0.022848,-0.018005,0.010508,-0.002388,-0.002167,0.008093,-0.024622,0.027185
avg_glucose_level,0.054722,0.238212,0.174540,0.161907,0.155329,-0.004783,1.000000,0.168913,0.131991,0.013908,-0.014665,0.016451,0.062770,-0.102195,-0.095035,0.067796,0.024011,0.017716
bmi,-0.025605,0.326189,0.160151,0.038865,0.335564,-0.000288,0.168913,1.000000,0.038917,0.077885,-0.028613,0.204055,0.070757,-0.442946,-0.265094,0.104675,0.106105,0.085518
stroke,0.009081,0.245109,0.127891,0.134905,0.108299,0.015415,0.131991,0.038917,1.000000,0.002660,-0.014885,0.011927,0.062150,-0.083888,-0.055924,0.064683,-0.004163,0.008920
work_type_Govt_job,-0.016598,0.130036,0.017661,0.001312,0.133964,0.013056,0.013908,0.077885,0.002660,1.000000,-0.025263,-0.444394,-0.167849,-0.151417,-0.097473,0.029865,0.046854,0.029994


In [17]:
#sepate labels and target
X = data.drop(columns = ['stroke'])
#target
Y = data['stroke']
#oversample data
smote = SMOTE(random_state=42)
X , Y = smote.fit_resample(X,Y)

before = data.stroke.value_counts(normalize=True)
after = Y.value_counts(normalize=True)
print('Rows before smote:' + ' {}'.format(data.shape[0]))
print('Rows after smote:' + ' {}'.format(X.shape[0]))

Rows before smote: 5109
Rows after smote: 9720


In [18]:
# separate into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,shuffle=True,random_state=10)

print('Train set shape:' + ' {}'.format(X_train.shape))
print('Test set shape:' + ' {}'.format(X_test.shape))

Train set shape: (7776, 17)
Test set shape: (1944, 17)


In [19]:
# Set up the subplots grid
fig = make_subplots(rows=1, cols=3, 
                    # Set the subplot titles
                    subplot_titles=['Age', 'Avg glucose level', 'BMI'])
#create boxplot visualization of numeric columns
fig.add_trace(go.Box(x=X_train.age, name='', showlegend=False), row=1, col=1)
fig.add_trace(go.Box(x=X_train.avg_glucose_level, name='', showlegend=False), row=1, col=2)
fig.add_trace(go.Box(x=X_train.bmi, name='', showlegend=False), row=1, col=3)
#config size
fig.update_layout(height=400,font_family='Verdana',paper_bgcolor='#edeae7',plot_bgcolor='#edeae7')
#show visualizations
fig.show()


In [20]:
from scipy import stats
u = data['bmi'].mean()  # 计算均值
std = data['bmi'].std()  # 计算标准差
stats.kstest(data['bmi'], 'norm', (u, std))

KstestResult(statistic=0.07322643012544339, pvalue=3.2063997298943748e-24)

In [21]:
#round decimals
X_train = round(X_train, 2)
X_test = round(X_test, 2)
#function to eliminate outliers base in Zscore
def Zscore_outlier(df):
    out=[]
    m = np.mean(df)
    sd = np.std(df)
    for i in df: 
        z = (i-m)/sd
        if np.abs(z) > 3: 
            out.append(i)
    out.sort()
    print(out)
Zscore_outlier(X_train['bmi'])
Zscore_outlier(X_test['bmi'])

[49.3, 49.3, 49.4, 49.5, 49.8, 49.9, 50.1, 50.2, 50.2, 50.2, 50.3, 50.5, 50.6, 50.6, 50.8, 51.05, 51.5, 51.7, 51.8, 51.82, 51.9, 51.9, 51.92, 52.33, 52.5, 52.7, 52.7, 52.73, 52.8, 52.8, 52.8, 52.82, 52.98, 53.4, 53.5, 53.8, 53.8, 53.9, 54.0, 54.0, 54.1, 54.2, 54.3, 54.31, 54.34, 54.6, 54.8, 54.89, 55.0, 55.0, 55.1, 55.2, 55.45, 55.7, 55.7, 55.7, 55.7, 55.9, 56.0, 56.0, 56.1, 56.12, 56.14, 56.6, 56.6, 57.2, 57.3, 57.7, 57.9, 59.7, 60.2, 60.9, 60.9, 61.2, 61.6, 63.3, 64.4, 66.8, 71.9, 92.0, 97.6]
[49.3, 49.5, 49.8, 49.8, 49.85, 50.1, 50.2, 50.3, 50.4, 50.9, 51.0, 52.3, 52.9, 53.4, 54.37, 54.6, 54.7, 54.7, 54.7, 55.9, 57.2, 57.5, 58.1, 64.8, 78.0]


In [22]:
#change outliers value to approximate maximum
X_train.loc[X_train.bmi >= u+3*std, 'bmi'] = u+3*std
X_test.loc[X_test.bmi >= u+3*std, 'bmi'] = u+3*std

In [23]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
X_train_std=std.fit_transform(X_train)
X_test_std=std.transform(X_test)